In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xyz = [[-875, 0, 9.755], [442, 0, 9.755], [1921, 0, 9.755], [2951, 0.5, 9.755], [-4132, 0.5, 23.618],
[-876, 0, 23.618]]
uv = [[76, 706], [702, 706], [1440, 706], [1867, 706], [264, 523], [625, 523]]

In [3]:
control_pt=np.array(xyz)
image_pt=np.array(uv)

In [4]:
def normalize(point):
    mean=np.mean(point,axis=0)
    s=2**0.5*point.shape[0]/np.sqrt(np.sum(np.square(point-mean)))
    
    if point.shape[1]==3:
        mat=[[s,0,0,-s*mean[0]],[0,s,0,-s*mean[1]],[0,0,s,-s*mean[2]],[0,0,0,1]]
    elif point.shape[1]==2:
        mat=[[s,0,-s*mean[0]],[0,s,-s*mean[1]],[0,0,1]]
    
    mat=np.array(mat)
    mat=np.linalg.inv(mat)
    comp_arr=np.concatenate((point,np.ones(shape=(point.shape[0],1))),axis=1)
    norm=np.dot(comp_arr,mat)
    return mat,norm

In [5]:
mat_3D,control_pt_norm=normalize(control_pt)
#control_pt_norm=control_pt_norm/control_pt_norm[:,-1].reshape(-1,1)
control_pt_norm= control_pt_norm[:,:-1]

mat_2D,image_pt_norm=normalize(image_pt)
#image_pt_norm=image_pt_norm/image_pt_norm[:,-1].reshape(-1,1)
image_pt_norm= image_pt_norm[:,:-1]

In [6]:
def calculate_dlt(pt_3,pt_2):
    A=[]
    for i in range(pt_3.shape[0]):
        arr1=[-pt_3[i][0],-pt_3[i][1],-pt_3[i][2],-1,0,0,0,0,pt_3[i][0]*pt_2[i][0],pt_3[i][1]*pt_2[i][0],pt_3[i][2]*pt_2[i][0],pt_2[i][0]]
        A.append(arr1)
        arr1=[0,0,0,0,-pt_3[i][0],-pt_3[i][1],-pt_3[i][2],-1,pt_3[i][0]*pt_2[i][1],pt_3[i][1]*pt_2[i][1],pt_3[i][2]*pt_2[i][1],pt_2[i][1]]
        A.append(arr1)
    
    A=np.array(A)
    
    U,sigma,V=np.linalg.svd(A)
    P=V[-1,:]
    H = P.reshape(3, pt_3.shape[1] + 1)
    H=np.dot(np.dot(np.linalg.inv(mat_2D),H),mat_3D)
    return H,sigma

In [7]:
Homo,sigma=calculate_dlt(control_pt_norm,image_pt_norm)

In [8]:
pt=np.concatenate((control_pt,np.ones(shape=(control_pt.shape[0],1))),axis=1)

In [9]:
np.sum(Homo[1]*pt[-1])/np.sum(Homo[-1]*pt[-1])

519.434195005755

In [10]:
H_inf=Homo[:,:3].reshape(-1,3)
h_vec=Homo[:,-1].reshape(-1,1)

In [11]:
X_o=np.dot(np.linalg.inv(H_inf),h_vec)

In [12]:
R_inv,K_inv=np.linalg.qr(np.linalg.inv(H_inf))

In [13]:
R=R_inv.T
K=np.linalg.inv(K_inv)
K=K/K[-1,-1]

In [14]:
H_inf_inv=np.linalg.inv(H_inf)

In [36]:
U_mat=np.zeros_like(H_inf_inv)
U_mat[:,0]=H_inf_inv[:,0]
for i in range(1,H_inf_inv.shape[1]):
    proj=np.dot(U_mat[:,:i].T,H_inf_inv[:,i].reshape(-1,1))
    proj=np.multiply(proj.T,U_mat[:,:i])/np.sum(U_mat[:,:i]**2,axis=0)
    U_mat[:,i]=H_inf_inv[:,i]-np.sum(proj,axis=1)
Q=U_mat/np.sqrt(np.sum(U_mat**2,axis=0))
R=np.dot(Q.T,H_inf_inv)

In [39]:
R

array([[ 5.77031464e+05, -7.37880360e+06,  3.03070605e+09],
       [-1.64655597e-08,  1.28244380e+04, -6.46111776e+06],
       [-2.67439754e-07,  4.48627120e-06,  1.04004501e+06]])

In [17]:
K_inv

array([[ 5.77031464e+05, -7.37880360e+06,  3.03070605e+09],
       [ 0.00000000e+00, -1.28244380e+04,  6.46111776e+06],
       [ 0.00000000e+00,  0.00000000e+00, -1.04004501e+06]])

In [37]:
Q

array([[-9.99999988e-01,  1.42174604e-04,  5.80330823e-05],
       [ 1.53562549e-04,  9.25841735e-01,  3.77911443e-01],
       [-3.96416711e-11,  3.77911447e-01, -9.25841746e-01]])

In [38]:
R_inv

array([[-9.99999988e-01, -1.42174604e-04, -5.80330818e-05],
       [ 1.53562549e-04, -9.25841735e-01, -3.77911442e-01],
       [-3.96416711e-11, -3.77911447e-01,  9.25841746e-01]])

In [40]:
H_inf_inv

array([[-5.77031457e+05,  7.37880534e+06, -3.03070687e+09],
       [ 8.86104225e+01,  1.07402920e+04, -5.12352462e+06],
       [-2.28744915e-05,  4.84650222e+03, -3.40464757e+06]])

In [41]:
np.dot(U_mat[:,:1].T,H_inf_inv[:,1].reshape(-1,1))

array([[-4.25780185e+12]])